# Näitekood Riigi Teataja pealkirja otsinguks

Päringulaiendaja kasutamine võimaldab meil teha päringuid, mis leiavad ülesse otsiõna kõik vormid eeldusel, et päringulaiendaja sisendandmebaas on ajakohane. Seetõttu on mõistlik teha otsing kahe tasemeline:

* päringulaiendajat kasutav otsing (kiire)
* kõiki sõnavorme pimedalt genereeriv otsing (aeglane)

Mõlema otsingu tegemiseks on tarvis kasutada Riigi Teataja veebiteenust, mis lubab otsida pealkirjadest täpseid sõnu.  

In [1]:
import math
import requests
import ipywidgets as widgets

from io import BytesIO
from pandas import concat
from pandas import read_csv
from pandas import DataFrame
from datetime import date

from typing import List, Union

## I. Päringulaiendaja liidestamine

In [2]:
def generate_worldforms_with_query_extender(words: Union[List[str], str],
                                            case: bool = True,
                                            search_sublemmas: bool = False):
    """
    Uses web service to get all wordforms existing in the documents for a list of lemmas.
    By default the extender keeps track of in which case the characters are, e.g. Euroopa and euroopa are different. 
    Returns a two column table with columns input, input_lemma, extension_type, score and search_wordform.
    The score shows the likelihood of the search wordform. The higher the better. 

    If the flag search_sublemmas is set returns also compund words containing.
    if the flag case is false extension is done in case insensitive manner.
    """
    words = [words] if isinstance(words, str) else words
    if not all(map(lambda x: x.find('\t') == -1, words)):
        raise ValueError("Inputs cannot contain '\\t' character. It corrupts the output")

    ANALYSER = "https://smart-search.tartunlp.ai/api/ea_paring_sl/tsv"
    HEADERS = {"Content-Type": "application/json; charset=utf-8"}
    POST_DATA_TEMPLATE = {"params":{"otsi_liitsõnadest": search_sublemmas}, "tss": "\t".join(words)}
    
    response = requests.post(ANALYSER, json=POST_DATA_TEMPLATE, headers=HEADERS)
    assert response.ok, "Webservice failed"

    return (read_csv(BytesIO(response.content), delimiter='\t')
            .rename(columns={
                'lemma': 'input_lemma', 
                'type': 'extension_type', 
                'confidence': 'score',
                'wordform': 'search_wordform'})
            [['input', 'input_lemma', 'extension_type', 'score', 'search_wordform']]
           )

### Standardnäited päringusõna laiendamisest

Vaikimisi laiendatakse otsisõna arvestades suurtäheortograafiat. Sealjuures võib astuste erinevus olla vaid õiges sisendlemmas samas kui laiendite hulk jääb samaks. 

In [3]:
display(generate_worldforms_with_query_extender('Euroopa'))
display(generate_worldforms_with_query_extender('euroopa'))
display(generate_worldforms_with_query_extender('EL'))
display(generate_worldforms_with_query_extender('el'))

,input,input_lemma,extension_type,score,search_wordform
0,Euroopa,Euroopa,word,820,Euroopa
1,Euroopa,Euroopa,word,819,euroopa


,input,input_lemma,extension_type,score,search_wordform
0,euroopa,Euroopa,word,820,Euroopa
1,euroopa,Euroopa,word,819,euroopa
2,euroopa,euroopa,word,819,euroopa


,input,input_lemma,extension_type,score,search_wordform
0,EL,EL,word,10,EL


,input,input_lemma,extension_type,score,search_wordform


Õige ja kirjavigase otsisõne väljund on sama kui vastavad lemmad langevad kokku. Samas võib juhtuda, et kirjaveaga vormi pole üldse sõnastikus.

In [4]:
display(generate_worldforms_with_query_extender('president'))
display(generate_worldforms_with_query_extender('bresident'))
display(generate_worldforms_with_query_extender('presidemt'))

,input,input_lemma,extension_type,score,search_wordform
0,president,president,word,18,presidendi
1,president,president,word,2,president


,input,input_lemma,extension_type,score,search_wordform
0,bresident,president,suggestion,18,presidendi
1,bresident,president,suggestion,2,president
2,bresident,president,compound,18,presidendi
3,bresident,president,compound,2,president


,input,input_lemma,extension_type,score,search_wordform


Lipu `search_sublemmas` sättimine võimaldab leida ka liitsõnu, mis sisaldavad otsisõna alamlemmana. 

In [5]:
display(generate_worldforms_with_query_extender('president', search_sublemmas=True))
display(generate_worldforms_with_query_extender('tuuma', search_sublemmas=True))
display(generate_worldforms_with_query_extender('aastane', search_sublemmas=True))
display(generate_worldforms_with_query_extender('19', search_sublemmas=True))

,input,input_lemma,extension_type,score,search_wordform
0,president,president,word,18,presidendi
1,president,president,word,2,president
2,president,president,compound,1,asepresidendi


,input,input_lemma,extension_type,score,search_wordform
0,tuuma,tuum,word,18,tuuma
1,tuuma,tuum,compound,1,tuumaavarii
2,tuuma,tuum,compound,2,tuumaavariist
3,tuuma,tuum,compound,1,tuumakahjustuste
4,tuuma,tuum,compound,1,tuumakütuse
5,tuuma,tuum,compound,4,tuumamaterjali
6,tuuma,tuum,compound,1,tuumaohutus
7,tuuma,tuum,compound,1,tuumaohutus
8,tuuma,tuum,compound,1,tuumaohutuse
9,tuuma,tuum,compound,2,tuumarelva


,input,input_lemma,extension_type,score,search_wordform
0,aastane,aastane,word,7,aastane
1,aastane,aastane,word,13,aastase
2,aastane,aastane,word,1,aastasele
3,aastane,aastane,word,1,aastases
4,aastane,aastane,word,3,aastasse
5,aastane,aastane,word,5,aastast
6,aastane,aastane,word,6,aastaste
7,aastane,aastane,word,3,aastastele
8,aastane,aastane,compound,1,19aastaste
9,aastane,aastane,compound,1,19aastastele


,input,input_lemma,extension_type,score,search_wordform
0,19,19,word,4,19
1,19,19,compound,1,19aastaste
2,19,19,compound,1,19aastastele


Erikujuliste nimede korral leitakse ülesse ka kõikvõimalikud tekstides olevad nime käändevormid.

In [6]:
display(generate_worldforms_with_query_extender('Strasbourg'))
display(generate_worldforms_with_query_extender('d’Ivoire'))
display(generate_worldforms_with_query_extender('ÜRO'))
display(generate_worldforms_with_query_extender('Meri'))

,input,input_lemma,extension_type,score,search_wordform


,input,input_lemma,extension_type,score,search_wordform


,input,input_lemma,extension_type,score,search_wordform
0,ÜRO,ÜRO,word,8,ÜRO


,input,input_lemma,extension_type,score,search_wordform
0,Meri,Meri,word,2,Meri
1,Meri,Meri,word,7,mere
2,Meri,Meri,word,1,mereni
3,Meri,Meri,word,1,meres
4,Meri,Meri,word,8,meri
5,Meri,Meri,word,1,merre


### Päringufraaside laiendamine

**Oluline:** Päringulaiendja ei tööta mitmesõnaliste pärisnimedega ja terminoloogia fraasidega. Seda eelkõige selle tõttu, et Riigi Teataja veebiteenus ei toeta fraaside otsimist. Teiseks on vastav fraaside normaliseerimismoodul Riigi Teataja kodulehel olemas. 


Fraaside laiendamise saaks lisada päringulaiendajasse, kuid see eeldaks fikseeritud fraaside tuvastamist tekstist. Selleks oleks vaja dokumentide lisaindekseerimisteenust, mis tuvastaks nimekirja alusel dokumendis olevaid fraase. Vastav laiendus eeldaks EstNLTK teegi kasutamist, mis on GPL v2 litsensiga. Me käsitleme seda eraldi osas [`01_dokumentide_indekseerimine`](../01_dokumentide_indekseerimine/). 

In [7]:
display(generate_worldforms_with_query_extender('Euroopa Liit'))
display(generate_worldforms_with_query_extender(['euroopa', 'liit']))

,input,input_lemma,extension_type,score,search_wordform


,input,input_lemma,extension_type,score,search_wordform
0,euroopa,Euroopa,word,820,Euroopa
1,euroopa,Euroopa,word,819,euroopa
2,euroopa,euroopa,word,819,euroopa
3,liit,liit,word,486,liidu
4,liit,liit,word,14,liiduga
5,liit,liit,word,15,liidule
6,liit,liit,word,17,liidult
7,liit,liit,word,7,liidus
8,liit,liit,word,1,liidust
9,liit,liit,word,36,liit


### Lühendite ja numbrite laiendamine

Lühendeid ja numbreid tüüpiliselt ei laiendata, sest need esinevad tekstides peamiselt käändumatus vormis. Samas on päringulaiendaja sisendandmebaasi võimalik laiendada nii, et lühendid laiendatakse täisnimeks või terminiks. Kui täisfraas on mitmesõnaline, siis antakse tagasi kõik selle fraasi esinemisvormid ainult siis, kui need on indekseerimies käigus eraldi peale märgitud. 


In [8]:
display(generate_worldforms_with_query_extender('19'))
display(generate_worldforms_with_query_extender('CO2'))
display(generate_worldforms_with_query_extender('COVID-19'))
display(generate_worldforms_with_query_extender('Valka-2'))
display(generate_worldforms_with_query_extender('TKNE-5'))

,input,input_lemma,extension_type,score,search_wordform
0,19,19,word,4,19


,input,input_lemma,extension_type,score,search_wordform
0,CO2,CO2,word,1,CO2


,input,input_lemma,extension_type,score,search_wordform
0,COVID-19,COVID-19,word,105,COVID-19


,input,input_lemma,extension_type,score,search_wordform
0,Valka-2,Valka-2,word,1,Valka-2


,input,input_lemma,extension_type,score,search_wordform
0,TKNE-5,TKNE-5,word,1,TKNE-5


## II. Riigi Teataja baasotsinguteenuse liidestamine 

Riigi Teataja dokumente on võimalik otsida läbi veebiteenuse, mis võimaldab otsida vaid täpseid sõnavorme. 

In [9]:
def search_rt_caption(wordform:str, match_type:str='koik_sonad', **kwargs):
    """
    Potential match types are: 
    * autocomplete – leiab aktid, mille pealkirjas on suvalises järjekorras sõnad, mis algavad otsitud sõnadega
    * koik_sonad – leiab aktid, mille pealkirjas esinevad kõik otsitud sõnad suvalises järjekorras
    * täpne – leiab aktid, mille pealkirjas esineb otsitud fraas
    """
    
    SEARCH_QUERY = 'https://www.riigiteataja.ee/api/oigusakt_otsing/1/otsi'

    # Let us pick all documents
    ARG_STRUCTURE = \
    {
        'leht': (int, 1),
        'limiit': (int, 500),
        'kehtiv': (date, None),
        'tulemused': (bool, True),
        'kehtivKehtetus': (bool, False),
        'mitteJoustunud': (bool, False),
        'kov': (bool, True),
        'dokument': (str, None)
    }

    # Check that keyword arguments are correct and have right type 
    payload = kwargs.copy() 
    for key, value in payload.items():
        arg_type, default_value = ARG_STRUCTURE.get(key, (None, None))
        if arg_type is None:
            raise ValueError(f'Unknown argument: {key}')
        elif not isinstance(value, arg_type):
            raise ValueError(f'Argument {key} must be of type {arg_type}')

    # Update important payload arguments        
    payload['leht'] = 1
    payload['kehtiv'] = date.today()
    payload['pealkiri'] = wordform
    payload['pealkiriOtsinguTyyp'] = match_type
    payload['filter'] = True
    payload['grupeeri'] = False
    payload['limiit'] = payload.get('limiit', 500)
        
    response = requests.get(SEARCH_QUERY, params=payload)
    assert response.status_code== 200, 'GET request failed'
    response = response.json()

    # Get the number of responce pages
    assert 'aktid' in response, 'Missing payload'
    assert 'metaandmed' in response, 'Missing meta field'
    assert 'kokku' in response['metaandmed'], 'Missing meta field'
    assert 'limiit' in response['metaandmed'], 'Missing meta field'

    total_count = response['metaandmed']['kokku']
    document_limit = response['metaandmed']['limiit']
    max_page = math.ceil(total_count/document_limit)

    if total_count == 0:
        return DataFrame(columns=['document_type', 'document_title', 'commencement_date', 'global_id'])
    
    # Iterate over responce sheets
    query_results = [None] * max_page 
    for page in range(max_page):

        payload['leht'] = page + 1
        response = requests.get(SEARCH_QUERY, params=payload)
        assert response.status_code== 200, 'GET request failed'
        response = response.json()

        document_count = len(response['aktid'])
        query_results[page] = DataFrame({
            'document_type': [None] * document_count,             
            'document_title': [None] * document_count, 
            'commencement_date': [None] * document_count,
            'global_id': [None] * document_count})

        for i, document in enumerate(response['aktid']):
            query_results[page].loc[i, 'global_id'] = document['globaalID']
            query_results[page].loc[i, 'document_title'] = document['pealkiri']
            query_results[page].loc[i, 'document_type'] = document['liik']
            query_results[page].loc[i, 'commencement_date'] = document['kehtivus'].get('algus')
        
    return (concat(query_results, axis=0)
            .sort_values(['document_type', 'document_title', 'commencement_date'], ascending=[True, True, False])
            .reset_index(drop=True))

Mitmesõnalise otsingusisendi korral annab see tagasi dokumendid, mille pealkirjades on kõik määratud sõnad
* Vaikimisi ei kontrollita nende järjestust.
* Täpse fraasi otsimiseks tuleb seada `match_type='täpne'`.

Tegelikult täpse fraasi otsing ei tööta ja kätte on võimalik saada vaid pealkirju, mis sisaldavad otsitud sõnu. See tähendab fraaside otsimisel palju valepositiivseid. Veelgi enam täpne otsing otsib fraasi ka sõnade seest.  

In [10]:
display(search_rt_caption('kodu- töökorra'))
display(search_rt_caption('kodu töökorra', match_type='täpne'))
display(search_rt_caption('odu töökorra', match_type='täpne'))

,document_type,document_title,commencement_date,global_id
0,seadus,Riigikogu kodu- ja töökorra seadus,2020-06-12,102062020009
1,seadus,Riigikogu kodu- ja töökorra seadus,2017-01-01,106052016012
2,seadus,Riigikogu kodu- ja töökorra seadus,2013-04-01,106072012014
3,seadus,Riigikogu kodu- ja töökorra seadus,2013-04-01,114092012003
4,seadus,Riigikogu kodu- ja töökorra seadus,2012-01-01,108072011070
5,seadus,Riigikogu kodu- ja töökorra seadus,2012-01-01,121032011033


,document_type,document_title,commencement_date,global_id
0,määrus,Õpilaskodu töökorralduse alused,2010-09-01,128092012016
1,määrus,Õpilaskodu töökorralduse alused,2010-09-01,13338320
2,seadus,Riigikogu kodu- ja töökorra seadus,2020-06-12,102062020009
3,seadus,Riigikogu kodu- ja töökorra seadus,2017-01-01,106052016012
4,seadus,Riigikogu kodu- ja töökorra seadus,2013-04-01,106072012014
5,seadus,Riigikogu kodu- ja töökorra seadus,2013-04-01,114092012003
6,seadus,Riigikogu kodu- ja töökorra seadus,2012-01-01,108072011070
7,seadus,Riigikogu kodu- ja töökorra seadus,2012-01-01,121032011033


,document_type,document_title,commencement_date,global_id
0,korraldus,Riigisaladuse kaitse komisjoni moodustamine ja...,2011-04-06,302072014008
1,määrus,Aegviidu valla tervisenõukogu moodustamine ja ...,2018-10-09,406102018038
2,määrus,Arstlike komisjonide moodustamise ja koosseisu...,2023-04-01,127012023028
3,määrus,Auditi komitee moodustamise ja töökorra põhimõ...,2010-01-01,13239422
4,määrus,Illuka Kooli hoolekogu moodustamise korra ja t...,2019-03-24,421032019002
5,määrus,Illuka Kooli hoolekogu moodustamise korra ja t...,2016-11-21,418112016023
6,määrus,Kaarma valla haridusasutuste hoolekogude moodu...,2016-03-19,416032016028
7,määrus,Kihelkonna Kooli hoolekogu moodustamise ja hoo...,2018-11-09,406112018045
8,määrus,"Kiviõli I Keskkooli, Kiviõli Vene Kooli ja Kiv...",2019-05-25,422052019017
9,määrus,"Kutsetegevuse valdkondade loetelu, kutsenõukog...",2015-07-18,115072015002


## III. Spellimisteenuse liidestamine

In [189]:
def spell_text(text: str):
    assert text.find(' ') == -1 and  text.find('\t') == -1, "Expecting a single word"

    # Webservice call
    SPELLER_QUERY = "https://smart-search.tartunlp.ai/api/speller/process"
    HEADERS = {"Content-Type": "application/json; charset=utf-8"}
    POST_DATA_TEMPLATE = {"content": text}
    response = requests.post(SPELLER_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
    assert response.ok, "Webservice failed"
    response = response.json()
    
    # Output reshaping
    token_count = len(response['annotations']['tokens'])
    tbl = DataFrame({'wordform': [None] * token_count, 'suggestion': [None] * token_count})
    for i, token in enumerate(response['annotations']['tokens']):
        features = token['features']
        tbl.loc[i, 'wordform'] = features['token']
        tbl.loc[i, 'suggestion'] = features.get('suggestions', features['token'])

    return tbl.explode('suggestion')

## III. Näiteotsingud päringulaiendajaga 

### Otsisõna 'president' 

Sõna president esineb pealkirjades kahes vormis ning ühe liitsõna osana.

In [12]:
df_style = [
    dict(selector="caption", props=[("text-align", "left"),("font-size", "150%"),("color", 'navy'), ("margin-top", "1em")])
]

display(generate_worldforms_with_query_extender('president', search_sublemmas=True)
        .style.set_caption("Otsisõna 'president' laiendused").set_table_styles(df_style))

display(search_rt_caption('president').style.set_caption("Baasotsingu 'president' vasted").set_table_styles(df_style))
display(search_rt_caption('presidendi').style.set_caption("Baasotsingu 'presidendi' vasted").set_table_styles(df_style))
display(search_rt_caption('asepresidendi').style.set_caption("Baasotsingu 'asepresidendi' vasted").set_table_styles(df_style))

,input,input_lemma,extension_type,score,search_wordform
0,president,president,word,18,presidendi
1,president,president,word,2,president
2,president,president,compound,1,asepresidendi


,document_type,document_title,commencement_date,global_id
0,määrus,Viimsi valla aukodaniku president Lennart Meri preemia taotlemise ja eraldamise kord,2023-10-22,419102023017
1,määrus,Viimsi valla aukodaniku president Lennart Meri preemia taotlemise ja eraldamise kord,2020-11-30,427112020027


,document_type,document_title,commencement_date,global_id
0,korraldus,Kaitseväe ja Kaitseliidu kasutamine turvalisuse tagamiseks Ameerika Ühendriikide presidendi visiidi ajal,2014-08-28,329082014008
1,korraldus,Vabariigi Presidendi ametihüve seaduse rakendamine,2021-11-26,330112021007
2,korraldus,Vabariigi Presidendi ametihüve seaduse rakendamine,2016-10-27,301112016009
3,korraldus,«Vabariigi Presidendi ametihüve seaduse» rakendamine,2008-06-05,12976818
4,määrus,Vabariigi Presidendi lähetuskulude hüvitamise korra kinnitamine,2023-02-24,121022023007
5,seadus,Riigikogu ja Vabariigi Presidendi poolt nimetatavate riigiametnike ametipalkade seadus,2014-01-01,129122012010
6,seadus,Riigikogu ja Vabariigi Presidendi poolt nimetatavate riigiametnike ametipalkade seadus,2013-01-01,128122011068
7,seadus,Riigikogu ja Vabariigi Presidendi poolt nimetatavate riigiametnike ametipalkade seadus,2012-01-01,13262582
8,seadus,Vabariigi Presidendi ametihüve seadus,2018-01-01,122062016013
9,seadus,Vabariigi Presidendi ametihüve seadus,2018-01-01,127062017010


,document_type,document_title,commencement_date,global_id
0,korraldus,Kaitseväe ja Kaitseliidu kaasamine avaliku korra kaitsesse Ameerika Ühendriikide asepresidendi visiidi ajal,2017-07-27,329072017002


### Otsisõna 'bresident'

Kuna otsisõna 'bresident' on kirjavigadega vorm, siis antakse välja laiendused tüübiga `suggestion`, mille hulgas pole liitsõnalaiendusi.

**TODO:** See on siin selge viga! Teenuses tuleb vastav viga ära parandada!

In [13]:
display(generate_worldforms_with_query_extender('bresident', search_sublemmas=False)
        .style.set_caption("Otsisõna 'bresident' laiendused").set_table_styles(df_style))

display(generate_worldforms_with_query_extender('bresident', search_sublemmas=True)
        .style.set_caption("Otsisõna 'bresident' laiendused").set_table_styles(df_style))

result = generate_worldforms_with_query_extender('bresident', search_sublemmas=True)
for query_string in list(set(result['search_wordform'])):
    display(search_rt_caption(query_string)
            .style.set_caption(f"Baasotsingu '{query_string}' vasted").set_table_styles(df_style))

,input,input_lemma,extension_type,score,search_wordform
0,bresident,president,suggestion,18,presidendi
1,bresident,president,suggestion,2,president
2,bresident,president,compound,18,presidendi
3,bresident,president,compound,2,president


,input,input_lemma,extension_type,score,search_wordform
0,bresident,president,suggestion,18,presidendi
1,bresident,president,suggestion,2,president


,document_type,document_title,commencement_date,global_id
0,korraldus,Kaitseväe ja Kaitseliidu kasutamine turvalisuse tagamiseks Ameerika Ühendriikide presidendi visiidi ajal,2014-08-28,329082014008
1,korraldus,Vabariigi Presidendi ametihüve seaduse rakendamine,2021-11-26,330112021007
2,korraldus,Vabariigi Presidendi ametihüve seaduse rakendamine,2016-10-27,301112016009
3,korraldus,«Vabariigi Presidendi ametihüve seaduse» rakendamine,2008-06-05,12976818
4,määrus,Vabariigi Presidendi lähetuskulude hüvitamise korra kinnitamine,2023-02-24,121022023007
5,seadus,Riigikogu ja Vabariigi Presidendi poolt nimetatavate riigiametnike ametipalkade seadus,2014-01-01,129122012010
6,seadus,Riigikogu ja Vabariigi Presidendi poolt nimetatavate riigiametnike ametipalkade seadus,2013-01-01,128122011068
7,seadus,Riigikogu ja Vabariigi Presidendi poolt nimetatavate riigiametnike ametipalkade seadus,2012-01-01,13262582
8,seadus,Vabariigi Presidendi ametihüve seadus,2018-01-01,122062016013
9,seadus,Vabariigi Presidendi ametihüve seadus,2018-01-01,127062017010


,document_type,document_title,commencement_date,global_id
0,määrus,Viimsi valla aukodaniku president Lennart Meri preemia taotlemise ja eraldamise kord,2023-10-22,419102023017
1,määrus,Viimsi valla aukodaniku president Lennart Meri preemia taotlemise ja eraldamise kord,2020-11-30,427112020027


### Otsisõna 'presidemt'

Päringulaiendajas ei ole võimalik käsitleda kõikvõimalikke kirjaveavorme. 
* Tehniliselt on uusi kirjaveatüüpe võimalik lisada veebiteenust `api_misspellings_generator` täiendades, aga kõikvõimalike kirjavea viise ei saa kunagi katta.
* Seetõttu tuleb otsingutes kasutada spellimisteenust kui alternatiivi päringulaiendaja kirjaveatuvastusele.
* Spelleri kasutamine annab välja kõik võimalikud sõnavormi korrektsioonid. Samas kui päringulaiendajas saab evatõenäolisemad vormid eemaldada. Seetõttu tuleks spellerit kasutada vaid siis, kui esialgne päringulaiendamistulemus on tühi. 

In [14]:
display(generate_worldforms_with_query_extender('presidemt', search_sublemmas=True)
        .style.set_caption("Otsisõna 'bresident' laiendused").set_table_styles(df_style))

result = spell_text('presidemt')
extensions = generate_worldforms_with_query_extender(list(set(result['suggestion'])), search_sublemmas=True)
display(extensions.style.set_caption("Otsisõna 'presidemt' laiendused kasutades spellerit").set_table_styles(df_style))

for query_string in list(set(extensions['search_wordform'])):
    display(search_rt_caption(query_string)
            .style.set_caption(f"Baasotsingu '{query_string}' vasted").set_table_styles(df_style))

,input,input_lemma,extension_type,score,search_wordform


,input,input_lemma,extension_type,score,search_wordform
0,president,president,word,18,presidendi
1,president,president,word,2,president
2,president,president,compound,1,asepresidendi


,document_type,document_title,commencement_date,global_id
0,korraldus,Kaitseväe ja Kaitseliidu kaasamine avaliku korra kaitsesse Ameerika Ühendriikide asepresidendi visiidi ajal,2017-07-27,329072017002


,document_type,document_title,commencement_date,global_id
0,korraldus,Kaitseväe ja Kaitseliidu kasutamine turvalisuse tagamiseks Ameerika Ühendriikide presidendi visiidi ajal,2014-08-28,329082014008
1,korraldus,Vabariigi Presidendi ametihüve seaduse rakendamine,2021-11-26,330112021007
2,korraldus,Vabariigi Presidendi ametihüve seaduse rakendamine,2016-10-27,301112016009
3,korraldus,«Vabariigi Presidendi ametihüve seaduse» rakendamine,2008-06-05,12976818
4,määrus,Vabariigi Presidendi lähetuskulude hüvitamise korra kinnitamine,2023-02-24,121022023007
5,seadus,Riigikogu ja Vabariigi Presidendi poolt nimetatavate riigiametnike ametipalkade seadus,2014-01-01,129122012010
6,seadus,Riigikogu ja Vabariigi Presidendi poolt nimetatavate riigiametnike ametipalkade seadus,2013-01-01,128122011068
7,seadus,Riigikogu ja Vabariigi Presidendi poolt nimetatavate riigiametnike ametipalkade seadus,2012-01-01,13262582
8,seadus,Vabariigi Presidendi ametihüve seadus,2018-01-01,122062016013
9,seadus,Vabariigi Presidendi ametihüve seadus,2018-01-01,127062017010


,document_type,document_title,commencement_date,global_id
0,määrus,Viimsi valla aukodaniku president Lennart Meri preemia taotlemise ja eraldamise kord,2023-10-22,419102023017
1,määrus,Viimsi valla aukodaniku president Lennart Meri preemia taotlemise ja eraldamise kord,2020-11-30,427112020027


### Otsisõna 'tuum'

See otsisõna esineb erinevates liitsõnades ja seega on vaja kindlasti määrata `search_sublemmas=True`.

In [15]:
display(generate_worldforms_with_query_extender('tuum', search_sublemmas=True)
        .style.set_caption("Otsisõna 'tuum' laiendused").set_table_styles(df_style))

extensions = generate_worldforms_with_query_extender('tuum', search_sublemmas=True)

for query_string in list(set(extensions['search_wordform'])):
    display(search_rt_caption(query_string)
            .style.set_caption(f"Baasotsingu '{query_string}' vasted").set_table_styles(df_style))

,input,input_lemma,extension_type,score,search_wordform
0,tuum,tuum,word,18,tuuma
1,tuum,tuum,compound,1,tuumaavarii
2,tuum,tuum,compound,2,tuumaavariist
3,tuum,tuum,compound,1,tuumakahjustuste
4,tuum,tuum,compound,1,tuumakütuse
5,tuum,tuum,compound,4,tuumamaterjali
6,tuum,tuum,compound,1,tuumaohutus
7,tuum,tuum,compound,1,tuumaohutus
8,tuum,tuum,compound,1,tuumaohutuse
9,tuum,tuum,compound,2,tuumarelva


,document_type,document_title,commencement_date,global_id
0,seadus,Tuumakahjustuste eest tsiviilvastutuse Viini konventsiooniga ühinemise seadus,2002-06-01,24627


,document_type,document_title,commencement_date,global_id
0,seadus,Tuumaavarii või kiirgusliku avariiolukorra puhul abi andmise konventsiooniga nõustumise seadus,2002-06-01,24631


,document_type,document_title,commencement_date,global_id
0,korraldus,Eesti Vabariigi valitsuse ja Soome Vabariigi valitsuse vahelise tuumaavariist või kiirgusohtlikust olukorrast operatiivse teatamise ning tuumaohutus- ja kiirguskaitsealase informatsiooni ja kogemuse vahetamise lepingu eelnõu heakskiitmine,1999-05-26,79214
1,seadus,Tuumaavariist operatiivse teatamise konventsiooniga nõustumise seadus,2002-06-01,24630


,document_type,document_title,commencement_date,global_id
0,korraldus,"""Põhja-Atlandi lepingu osalisriikide tuumateabealase koostöö kokkuleppe"" heakskiitmine ja volituste andmine",2005-05-10,898590


...

,document_type,document_title,commencement_date,global_id
0,korraldus,Tuumarelva leviku tõkestamise lepingu III artikli lõigete 1 ja 4 rakendamiseks sõlmitud kokkuleppega ja selle lisaprotokolliga ühinemine,2005-10-10,954451


,document_type,document_title,commencement_date,global_id
0,korraldus,«Eesti Vabariigi valitsuse ja Rahvusvahelise Aatomienergiaagentuuri vahelise kokkuleppe kaitseabinõude rakendamise kohta seoses tuumarelvade leviku tõkestamise lepinguga» eelnõu heakskiitmine,1997-11-06,74247


,document_type,document_title,commencement_date,global_id
0,seadus,Tuumarelvakatsetuste üldise keelustamise lepingu ratifitseerimise seadus,2002-06-01,26008


,document_type,document_title,commencement_date,global_id
0,seadus,Tuumaohutuse konventsiooniga ühinemise seadus,2005-12-18,964508


,document_type,document_title,commencement_date,global_id


### Otsisõnd 'Strasbourg' ja 'Tammsaare'

**TODO:** Ei tööta millegipärast


In [16]:
display(generate_worldforms_with_query_extender('Strasbourg', search_sublemmas=True)
        .style.set_caption("Otsisõna 'Strasbourg' laiendused").set_table_styles(df_style))

display(generate_worldforms_with_query_extender('Tammsaare', search_sublemmas=True)
        .style.set_caption("Otsisõna 'Tammsaare' laiendused").set_table_styles(df_style))

extensions = generate_worldforms_with_query_extender('Tammsaare', search_sublemmas=True)

for query_string in list(set(extensions['search_wordform'])):
    display(search_rt_caption(query_string)
            .style.set_caption(f"Baasotsingu '{query_string}' vasted").set_table_styles(df_style))

,input,input_lemma,extension_type,score,search_wordform


,input,input_lemma,extension_type,score,search_wordform
0,Tammsaare,Tammsaare,word,6,Tammsaare
1,Tammsaare,Tammsaare,word,2,Tammsaarele


,document_type,document_title,commencement_date,global_id
0,korraldus,Sihtasutuse A. H. Tammsaare Muuseum Vargamäel asutamine,2016-02-11,316022016012
1,määrus,A. H. Tammsaare nimelise Järva valla kirjanduspreemia statuut,2021-10-08,405102021015
2,määrus,A. H. Tammsaare tee nime kasutusala pikendamine,1996-06-28,419092013013
3,määrus,Pärnu Tammsaare Kooli põhimäärus,2020-01-13,410012020018
4,määrus,Pärnu Tammsaare Kooli põhimäärus,2020-01-13,410012020007
5,määrus,Pärnu Tammsaare Lasteaia põhimäärus,2016-10-07,404102016021


,document_type,document_title,commencement_date,global_id
0,korraldus,Rein Tammsaarele ja tema perekonnaliikmetele diplomaatilise passi andmine,2011-08-11,315082011015
1,korraldus,Rein Tammsaarele ja temaga kaasas olevatele perekonnaliikmetele diplomaatilise passi andmine,2014-07-24,329072014009


## IV. Lõplik otsimisteenuse loogika ja vastav kasutajaliidese näide 

Lihtsuse ja selguse mõttes realiseerime lõpliku otsimisteenuse Jypyteri widget-ina. 

* Selle kasutusloogika on täpselt analoogne veebilehe elementidega
* Samas saab seda siin esitada Pyhtoni funktsioonide abil
* Selle üleviimine JavaScripti peaks olema elementaarne.

Intuitiivse kasutajakogemuse loomine on tegelikult keeruline: 

* Selleks peaks widget interakteeruma kasutajaga ja informeerima teda erinevatest otsustest.
* Ühesõnalise päringu korral on selge, mis võiks juhtuda.
* Mitmesõnalise päringu korral muutub see mittetriviaalseks:
  * Mida teha puuduvate sõnadega otsingus ( presidendi rohekaskollane ametiraha)
  * Mida teha valesti kirjutatud sõnadega otsingus (töö- jja puhkeaja)
  * Mida teha osassõnade otsinguga (tuumarelva üldine keelustamine, kus tekstis on tuumarelvakatsetuste)
  * Kas kogu fraas peab olema järjestikkune või võib seal vahel ka sõnu olla

Siin teeme kõige mitmesõnaliste otsifraaside korral kõige lihtsama lahenduse:
* Vaatame iga sõna laiendusi eraldi ning moodustame vastuse ühisosana.


In [348]:
# Widget layout with minimal styling
search_text_label = widgets.Label('Lihtne otsing seaduste ja üleriigiliste määruste pealkirjadest')

search_text = widgets.Text(
    description='',
    placeholder='presidendi ametiraha',
    value = 'Euroopa Liit',
    layout=widgets.Layout(width='90%')
)

search_button = widgets.Button(
    description="Otsi", 
    layout=widgets.Layout(left='35%', width='20%'))

search_widget = widgets.VBox(
    [search_text_label, search_text, search_button], 
    layout=widgets.Layout(width='90%', padding='100px'))

query_output = widgets.Output()

df_style = [dict(selector="caption", props=[("text-align", "left"),("font-size", "150%"),("color", 'navy'), ("margin-top", "1em")])]

def query_expansion(arg):
    query_output.clear_output()
    input_text = str(search_text.value).split(' ')

    # The simple case of a single searchword
    if len(input_text) == 1:
        result = generate_worldforms_with_query_extender(input_text, search_sublemmas=True)

        # Exact match 
        search_words = set(result.loc[result['extension_type'] == 'word', 'search_wordform'])

        with query_output:
            if len(search_words) != 0:
                print('Täpsed vasted')
            for query_string in search_words:
                display(search_rt_caption(query_string).style
                        .set_caption(f"Baasotsingu '{query_string}' vasted").set_table_styles(df_style))
            
        # Compounds
        search_words = set(result.loc[result['extension_type'] == 'compound', 'search_wordform'])

        with query_output:
            if len(search_words) != 0:
                print('Liitsõnade osad')
            for query_string in search_words:
                display(search_rt_caption(query_string).style
                        .set_caption(f"Baasotsingu '{query_string}' vasted").set_table_styles(df_style))

        # Abort if exact match was found
        if any(result['extension_type'] != 'suggestion'):
            return

        # Suggestions
        search_words = set(result.loc[result['extension_type'] == 'suggestion', 'search_wordform'])

        with query_output:
            if len(search_words) != 0:
                print('Sellist otsisõna pole, vaatame läbi soovitusi')
                display(result.loc[result['extension_type'] == 'suggestion', ['input', 'input_lemma']].style
                        .set_caption(f"Otsisõnale lähimad terminid").set_table_styles(df_style))
            for query_string in search_words:
                display(search_rt_caption(query_string).style
                        .set_caption(f"Baasotsingu '{query_string}' vasted").set_table_styles(df_style))

        
        # Fallback to direct spelling
        if len(result) == 0:
            suggestions = spell_text(input_text[0])
            
            if (len(suggestions) == 0) or (len(suggestions) == 1) and (suggestions.loc[0,'suggestion'] is None):
                with query_output:
                    print(f"Otsingule {input_text[0]} ei leitud ühtegi vastust")
                    return

            with query_output:
                print('Sellist otsisõna pole, vaatame läbi soovitusi')
                display(suggestions.style
                        .set_caption(f"Otsisõnale lähimad terminid").set_table_styles(df_style))
            
            refined_result = generate_worldforms_with_query_extender(list(set(suggestions['suggestion'])), search_sublemmas=True) 

            # Non-compounds
            search_words = set(refined_result.loc[refined_result['extension_type'] != 'compound', 'search_wordform'])
                
            with query_output:
                for query_string in search_words:
                    display(search_rt_caption(query_string).style
                            .set_caption(f"Baasotsingu '{query_string}' vasted").set_table_styles(df_style))
                    

            # Compounds
            search_words = set(refined_result.loc[refined_result['extension_type'] == 'compound', 'search_wordform'])
    
            with query_output:
                if len(search_words) != 0:
                    print('Liitsõnade osad')
                for query_string in search_words:
                    display(search_rt_caption(query_string).style
                            .set_caption(f"Baasotsingu '{query_string}' vasted").set_table_styles(df_style))
        
            
            return 

    # We have a list of search words
    else:

        # It is comfusing to search subwords from phrase
        result = generate_worldforms_with_query_extender(input_text, search_sublemmas=False)
        search_words = result.groupby('input', as_index=False).aggregate(search_words=('search_wordform', set))

        # Lets ignore the fact that some words have multiple lemmas
        if len(search_words) != len(input_text):
            with query_output:
                print('Mõned otsisõnad pole andmebaasis jätkame nendega, mis on')

        matches = [None] * len(search_words)
        for i,row in search_words.iterrows():
            matches[i] = concat([search_rt_caption(word) for word in row['search_words']], axis=0).drop_duplicates().set_index('global_id')

        with query_output:
            print(f"Otsisõnu {list(search_words['input'])} vorme sisaldavad pealkirjad")
            display(concat(matches, axis=1, join='inner'))        
    
    

# Add event handlers
search_button.on_click(query_expansion)

# Form the final widget
widgets.VBox([search_widget, query_output])

## V. Märkused JavaScriptiga liidestamiseks

Ülaltoodud näiderakendus oli ehitatud kasutades veebiteenusega liidestumiseks Pyhton-i `pandas` paketi andmestruktuuri `DataFrame`.
Veebilehtede loomisel on mõistlik kasutada liidestamiseks Javascripti JSON objekte. Selleks tuleb päringulaiendaja teenuse
```
https://smart-search.tartunlp.ai/api/ea_paring_sl/tsv
```
asemel kasutada `json` lõppkunktile vastavad URL-i
```
https://smart-search.tartunlp.ai/api/ea_paring_sl/json
```
ning teha somu operatsioone JSON puuga.

### Kuidas muuta päringuid efektiivsemaks

* Kõige lihtsam viis rakenduse tööaja vähendamiseks on lubada Riigi Teataja otsinguteenuses OR-päringuid
* Alternatiiviks on lisada sõnavormide genereerimine Riigi Teataja otsingu teenusesse nii, et päringule 
  vastatakse kõikide otsisõna vormidega (president -> president, presidendi,...) tehes teenuse siseselt vastavad päringud.
* Sellest veel parem on indekseerida dokumendid sõna algvormidega ja teha otsingut üle nende.